In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets list -s arxiv

ref                                                        title                                              size  lastUpdated          downloadCount  
---------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
Cornell-University/arxiv                                   arXiv Dataset                                     902MB  2020-11-08 00:52:17           4030  
neelshah18/arxivdataset                                    ARXIV data from 24,000+ papers                     18MB  2018-03-31 03:47:25           1696  
prasunroy/natural-images                                   Natural Images                                    342MB  2018-08-11 18:24:11          10490  
rmisra/news-headlines-dataset-for-sarcasm-detection        News Headlines Dataset For Sarcasm Detection        3MB  2019-07-03 23:52:57          19379  
tayorm/arxiv-papers-metadata                               Arxiv Papers Metadata D

In [ ]:
!kaggle datasets download Cornell-University/arxiv

 99% 897M/902M [00:36<00:00, 27.7MB/s]
100% 902M/902M [00:36<00:00, 26.0MB/s]


In [ ]:
!unzip /content/arxiv.zip

Archive:  /content/arxiv.zip
  inflating: arxiv-metadata-oai-snapshot.json  


In [ ]:
!python -m pip install dask[complete]

     |████████████████████████████████| 92kB 2.3MB/s 
     |████████████████████████████████| 665kB 8.2MB/s 
     |████████████████████████████████| 102kB 7.2MB/s 
  Created wheel for locket: filename=locket-0.2.0-cp36-none-any.whl size=4040 sha256=e7787fd29e32804169327c132956bba7b051a50006937f91cdd74c96045f8b52
  Stored in directory: /root/.cache/pip/wheels/26/1e/e8/4fa236ec931b1a0cdd61578e20d4934d7bf188858723b84698
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=e1e47773dd6f56e26fb15b8ef8fcf49791ad2e26314b6b448cb2408ab116935b
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built locket contextvars
ERROR: distributed 2.30.1 has requirement cloudpickle>=1.5.0, but you'll have cloudpickle 1.3.0 which is incompatible.
  Found existing installation: distributed 1.25.3
    Uninstalling distributed-1.25.3:
      Successfully uninstalled distributed-1.25.3


In [ ]:
import json
import pandas as pd
#import dask as dd
#import dask.bag as db

In [ ]:
file = db.read_text("/content/arxiv-metadata-oai-snapshot.json").map(json.loads)
file

dask.bag<loads, npartitions=1>

In [ ]:
def flatten(record):
    return {
        'id':record['id'],
        'title':record['title'],
        'categories':record['categories'],
        'abstract':record['abstract']
    }

In [ ]:
df = file.map(flatten).to_dataframe()

In [ ]:
df = df.compute()

In [ ]:
df.to_csv("arxiv_classification_dataset.csv",single_file = True)

['/content/arxiv_classification_dataset.csv']

In [ ]:
type(df)

pandas.core.frame.DataFrame

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/arxiv_classification_dataset.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
def get_categories_count(column):
    
    categories = column.values
    categories_count = {}
    distinct_categories_count = {}
    categories_count = {label:categories_count.get(label,0)+1 for label in categories}
    for label in categories:
        categories_count[label] = categories_count.get(label,0) + 1
        for distinct_label in label.split():
            distinct_categories_count[distinct_label] = distinct_categories_count.get(distinct_label,0) + 1
    return categories_count,distinct_categories_count

In [ ]:
def proportionate_stratified_random_sampling(df,type,nrows = None,sampling_percentage = None):
    sample_df = pd.DataFrame(columns=df.columns)
    sample_list = []
    print(sample_df.columns)
    if type == 'multilabel':
        sampled_indices = set()
        categories_count, distinct_categories_count = get_categories_count(df['categories'])
        print(distinct_categories_count)
        if sampling_percentage == None:
            sampling_percentage = (nrows * 100 / len(df))/100
        for category in distinct_categories_count.keys():
            temp_df = df[(df['categories'].str.contains(category)) & (~(df.index.isin(sampled_indices)))]
            temp_df = temp_df.sample(frac = sampling_percentage)
            if len(temp_df)!=0:
                sample_list.append(temp_df)
                sampled_indices.union( set(sample_df.index.values.tolist()))

    return pd.concat(sample_list)


In [ ]:
df['categories'] = df['categories'].apply(lambda x:x.split())
df['categories'] = df['categories'].apply(lambda x:list(set([item.split('-',1)[0] for item in x ])))
df['categories'] = df['categories'].apply(lambda x:list(set([item.split('.',1)[0] for item in x ])))
df['categories'] = df['categories'].apply(lambda x:' '.join(x))

In [ ]:
categories_count, distinct_categories_count = get_categories_count(df['categories'])

In [ ]:
top_11_categories_excluded = [item[0] for item in sorted(distinct_categories_count.items(), key =lambda kv:(kv[1], kv[0]))][:20]
top_11_categories_excluded

['bayes',
 'ao',
 'plasm',
 'acc',
 'atom',
 'supr',
 'comp',
 'chem',
 'mtrl',
 'funct',
 'adap',
 'patt',
 'dg',
 'cmp',
 'solv',
 'alg',
 'chao',
 'econ',
 'eess',
 'nlin']

In [ ]:
df['categories'] = df['categories'].apply(lambda x:x.split())
subdf = df[~pd.DataFrame(df.categories.tolist()).isin(top_11_categories_excluded).any(1)]

In [ ]:
subdf

,Unnamed: 0,id,title,categories,abstract
0,0,704,Calculation of prompt diphoton production cros...,[hep],A fully differential calculation in perturba...
1,1,704,Sparsity-certifying Graph Decompositions,"[math, cs]","We describe a new algorithm, the $(k,\ell)$-..."
2,2,704,The evolution of the Earth-Moon system based o...,[physics],The evolution of Earth-Moon system is descri...
3,3,704,A determinant of Stirling cycle numbers counts...,[math],We show that a determinant of Stirling cycle...
4,4,704,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,[math],In this paper we show how to compute the $\L...
...,...,...,...,...,...
1770119,1770119,quant-ph/9912118,A Fast and Compact Quantum Random Number Gener...,[quant],We present the realization of a physical qua...
1770120,1770120,quant-ph/9912119,Nuclear Teleportation,[quant],"Until recently, only science-fiction authors..."
1770122,1770122,quant-ph/9912121,Simple pulse sequence for quantum logic operat...,[quant],This paper has been withdrawn by the authors...
1770123,1770123,quant-ph/9912122,Optimal signal ensembles,[quant],Classical messages can be sent via a noisy q...


In [ ]:
subdf['categories'] = subdf['categories'].apply(lambda x:' '.join(x))
categories_count, distinct_categories_count = get_categories_count(subdf['categories'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
distinct_categories_count

{'astro': 276913,
 'cond': 287416,
 'cs': 279074,
 'gr': 83567,
 'hep': 296710,
 'math': 457166,
 'nucl': 59723,
 'physics': 168028,
 'q': 41693,
 'quant': 104875,
 'stat': 75418}

In [ ]:
sample_df = proportionate_stratified_random_sampling(subdf,'multilabel',nrows=110000)

Index(['Unnamed: 0', 'id', 'title', 'categories', 'abstract'], dtype='object')
{'hep': 296710, 'math': 457166, 'cs': 279074, 'physics': 168028, 'cond': 287416, 'gr': 83567, 'astro': 276913, 'quant': 104875, 'q': 41693, 'nucl': 59723, 'stat': 75418}


In [ ]:
sample_df

,index,Unnamed: 0,id,title,categories,abstract
0,1558996,1558996,hep-ph/0005099,Plasma Energy Loss into Kaluza-Klein Modes,hep astro,"Recently, Barger {\em et al.} computed energ..."
1,878882,878882,1708.04,Gravitational closure of weakly birefringent e...,gr hep,We derive the gravitational dynamics of the ...
2,1725923,1725923,nucl-th/9510062,Parity Violation and the Nucleon-Nucleon System,nucl hep,The status of the field of nuclear parity vi...
3,386989,386989,1211.47,Revisiting the gravitino dark matter and baryo...,hep astro,We reinvestigate the scenario that the amoun...
4,1656263,1656263,math-ph/0411072,Algebraic approach to Quantum Field Theory,gr math hep,We provide a short introduction to the main ...
...,...,...,...,...,...,...
150612,341747,341747,1205.27,A Multivariate Graphical Stochastic Volatility...,stat,The Gaussian Graphical Model (GGM) is a popu...
150613,318069,318069,1201.61,A robust and sparse K-means clustering algorithm,stat,In many situations where the interest lies i...
150614,1000752,1000752,1807.04,Near-infrared Mira Period-Luminosity Relations...,stat astro,We analyze sparsely-sampled near-infrared (J...
150615,1201555,1201555,1911.03,Improved Visual Localization via Graph Smoothing,stat cs,Vision based localization is the problem of ...


In [ ]:
sample_df = sample_df.sample(150000)

In [ ]:
categories_count, distinct_categories_count = get_categories_count(subdf['categories'])
distinct_categories_count

{'astro': 276913,
 'cond': 287416,
 'cs': 279074,
 'gr': 83567,
 'hep': 296710,
 'math': 457166,
 'nucl': 59723,
 'physics': 168028,
 'q': 41693,
 'quant': 104875,
 'stat': 75418}

In [ ]:
#sample_df.drop_duplicates(subset=['abstract'],inplace=True)
sample_df.reset_index(inplace=True)
sample_df.to_csv("/content/drive/MyDrive/arxiv_sample_final4.csv",index=False)

Index(['id', 'title', 'categories', 'abstract'], dtype='object')
{'hep': 297543, 'math': 458395, 'cs': 301330, 'physics': 162153, 'cond': 282717, 'gr': 84116, 'astro': 278635, 'nucl': 60040, 'quant': 105561, 'stat': 76276, 'eess': 26807}


In [ ]:
sample_df

,level_0,index,Unnamed: 0,id,title,categories,abstract
0,98662,1438735,1438735,cond-mat/0002333,Inhomogeneous order-parameter suppression in d...,cond,We report on a numerical study of disorder e...
1,115006,1384384,1384384,astro-ph/0310023,Doppler-like effect and doubtful expansion of ...,astro,"The distance contraction, as observed in ele..."
2,105193,473574,473574,1310.86,Probing a cosmological model with a $\Lambda =...,gr astro,In this work we study the evolution of matte...
3,149192,415836,415836,1303.42,Additive inverse regression models with convol...,stat math,In a recent paper Birke and Bissantz (2008) ...
4,145071,707777,707777,1602.07,Effective potential from zero-momentum potential,nucl physics,We obtain the centre-of-mass frame effective...
...,...,...,...,...,...,...,...
149995,93319,686594,686594,1512.04,From Chemistry to Functionality: Trends for th...,cond,We present a systematic ab-initio study of t...
149996,72918,848171,848171,1705.05,Forecasting using incomplete models,cs,We consider the task of forecasting an infin...
149997,30314,182107,182107,1004.02,Any discrete almost periodic set of finite typ...,math,A discrete set in the Euclidian space is alm...
149998,108471,107503,107503,902.129,Discontinuous Galerkin method for computing gr...,gr,Gravitational wave emission from extreme mas...
